# Imports

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Datos

In [71]:
# Cargar dataframe
df_original = pd.read_csv('../input/age-gender-and-ethnicity-face-data-csv/age_gender.csv')

In [72]:
df = df_original.copy()
df["pixels"] = df["pixels"].apply(lambda x: np.array(x.split(" "), dtype=np.float32).reshape(48, 48, 1))

In [73]:
df["pixels"][0].shape

In [74]:
fig, axes = plt.subplots(1, 4, figsize=(10, 5))

for i in range(4):
    random_face = np.random.choice(len(df))
    axes[i].set_title('Age: {0}, Sex: {1}'.format(df['age'][random_face], df['gender'][random_face]  ))
    axes[i].imshow(df['pixels'][random_face])
    axes[i].axis('off')

### Extraer solo datos necesarios

In [75]:
images = np.array(df["pixels"].to_list())
genders = np.array(df["gender"].to_list())

In [76]:
print("Shape of images: {}, Shape of genders: {}".format(images.shape, genders.shape))

# Preparación Red Neuronal

In [77]:
from sklearn.model_selection import train_test_split
images_train, image_test, genders_train, genders_test = train_test_split(images, genders, test_size=0.3, random_state=44)

In [78]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator

train_datagenerator = ImageDataGenerator(rescale=1/255)
train_generator = train_datagenerator.flow(
    images_train, genders_train, batch_size=32
)

test_datagenerator = ImageDataGenerator(rescale=1/255)
test_generator = test_datagenerator.flow(
    image_test, genders_test, batch_size=32
)

In [79]:
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint_filepath="./callback/ALLmodel_weights.h5"
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

earlystop = EarlyStopping(patience=6)
learning_rate_reduction = ReduceLROnPlateau(
    monitor='val_acc',
    patience= 3,
    verbose=1,
)

callbacks = [earlystop, learning_rate_reduction, checkpoint]

In [80]:
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Flatten,MaxPooling2D,Conv2D,Dropout

inputs = Input(shape=(48, 48, 1))

x = Conv2D(32, (3, 3), activation='relu', padding = 'same')(inputs)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding = 'same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding = 'same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.2)(x)
x = Conv2D(128, (3, 3), activation='relu', padding = 'same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.2)(x)
x = Flatten()(x)
x = Dropout(0.5)(x)

outputs = Dense(1, activation='relu')(x)

model:Model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="Adam", loss="mean_squared_error", metrics=["accuracy"])

model.summary()

In [210]:
model.fit(
    train_generator, 
    epochs = 15,
    steps_per_epoch = train_generator.n // train_generator.batch_size,
    validation_data = test_generator,
    validation_steps = test_generator.n // test_generator.batch_size,
)

In [82]:
results = pd.DataFrame(model.history.history)

In [83]:
results[['loss','val_loss']].plot(title='Model Loss')
plt.savefig('./figures/loss.png')

In [84]:
results[['accuracy','val_accuracy']].plot(title='Model Acurracy')
plt.savefig('./figures/accuracy.png')

In [ ]:
# Mostrar el accuracy y val_accuracy obtenidos
print("Accuracy: {}, Val_Accuracy: {}".format(results['accuracy'].iloc[-1], results['val_accuracy'].iloc[-1]))
# Mostrar el loss y val_loss obtenidos
print("Loss: {}, Val_Loss: {}".format(results['loss'].iloc[-1], results['val_loss'].iloc[-1]))

In [219]:
# Hacer confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay
# concatenate n batches

class_names = ["Hombre", "Mujer"]
n = 10

image_batch, classes_batch = next(test_generator)

for batch in range(n):
    temp = next(test_generator)
    image_batch = np.concatenate((image_batch, temp[0]))
    classes_batch = np.concatenate((classes_batch, temp[1]))

classes_batch = classes_batch.tolist()
y_predict = model.predict(image_batch).reshape(32*(n+1))
                                               
y_temp = []
for x in y_predict:
    x = float(x)
    y_temp.append(0 if x < 0.5 else 1)
y_predict = y_temp

ConfusionMatrixDisplay.from_predictions(
    y_true = classes_batch,
    y_pred = y_predict,
    display_labels=class_names,
    cmap='Blues'
)
                                               
plt.show()

In [ ]:
def predict_one(model, num_images = None):  
    image_batch, classes_batch = next(test_generator)
    predicted_batch = model.predict(image_batch)
    for k in range(0,image_batch.shape[0] if num_images is None else num_images):
        image = image_batch[k]
        real_class = class_names[0 if classes_batch[k] < 0.5 else 1]
        predicted_class = class_names[0 if predicted_batch[k] < 0.5 else 1]
        value_predicted = predicted_batch[k]
        isTrue = (real_class == predicted_class)
        plt.figure(k)
        plt.title(str("Predicción Correcta" if isTrue else "Predicción Incorrecta") + ' - class: ' + real_class + ' - ' + 'predicted: ' + predicted_class + str(value_predicted))
        plt.axis('off')
        plt.savefig('./figures/' + real_class + '_' + predicted_class + '_' + str(value_predicted) + '.png')
        plt.imshow(image)

In [86]:
predict_one(model, 5)